In [4]:
import requests
import pandas as pd
from tqdm import tqdm


from datetime import datetime
import logging
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%Y%m%d%H%M%S")
logging.basicConfig(filename=f'scrapping_logs/run_betexplorer_scrapping_{timestampStr}.log',  level=logging.DEBUG,
    format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
timestampStr

'20221209160643'

In [5]:
logging.info('INITIALISING ....')

In [6]:
import bs4 as BeautifulSoup
from bs4 import BeautifulSoup

In [7]:

wcs=[]
for y in range(2022,2023):
    #for m in range(1,10):
    #u = f"https://www.betexplorer.com/soccer/world/friendly-international-{y}/results/?month={y}-0{m}"
    u = f"https://www.betexplorer.com/soccer/england/premier-league-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/england/championship-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/england/league-one-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/france/ligue-1-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/france/ligue-2-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/germany/bundesliga-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/italy/serie-b-{y-1}-{y}/results/"
    wcs.append(u)
    u = f"https://www.betexplorer.com/soccer/italy/serie-a-{y-1}-{y}/results/"
    wcs.append(u)



#wcs= ['https://www.betexplorer.com/soccer/world/world-cup-2022/']


# https://www.betexplorer.com/soccer/world/friendly-international-2013/results/?month=2013-02
urls = []
games_bets_rows = []
for wc in tqdm(wcs):
    url = wc
    #print(wc)
    try:
        page = requests.get(url)

        content = page.content
        soup = BeautifulSoup(page.content, 'html.parser')
        tr_rows = soup.find_all('tr')


        ix = 0
        for row in tr_rows:
            #print(ix)
            ix=ix+1
            r = str(row)
            #print(r)

            if 'in-match' in r:
                logging.info(r)
                matchday = r.split('<td class="h-text-right h-text-no-wrap">')[1]

                temp = BeautifulSoup(r, 'html.parser')
                spans = temp.find_all('span')
                i=0
                for span in spans:
                    if i < 2:

                        s = str(span)
                        s = s.replace('<','').replace('>','').replace('strong','').replace('span','').replace('/','')
                        logging.info(f'      {s}')
                        if i == 0:
                            home_team=s
                        else:
                            away_team=s
                        i=i+1


                odds = temp.find_all('td', class_='table-main__odds')
                input_tag = temp.find("td", {"name":"City"})
                odds_list=[]
                valid_odds=False
                for odd in odds:
                    o = str(odd).split('data-odd=')
                    logging.info(o[1].split('"')[1])
                    odds_list.append(o[1].split('"')[1])
                    valid_odds=True

                games = temp.find_all('td', class_='h-text-center')
                for game in games:
                    gam= str(game)
                    #logging.info(gam)
                    # get the game + key
                    g = gam.split('href="')[1].split('">')[0]
                    g = g.split('/')
                    g = f"{g[1]}/{g[2]}/{g[3]}/{g[4]}-{g[5]}"
                    # get game score
                    s = gam.split('</a')[0]
                    s = s.split('/">')[1]
                    s = s.split(' ')[0]
                    if ':' in s:
                        logging.info(s)
                        correct_score=s
                        game_entry=g
                        game_url = url
                        urls.append([g,url,s])
                logging.info(matchday.split('<')[0])
                matchday = matchday.split('<')[0]

                if valid_odds:
                    games_bets_rows.append([game_entry, matchday, home_team, away_team, correct_score, odds_list[0], odds_list[1], odds_list[2], game_url ])


    except Exception as e:
        logging.info(f'no-game - {e}')
        #print('no Game')
        #print(e)
        #print(soup)

        
df = pd.DataFrame(games_bets_rows)
df.columns = ['oddsportal','matchday','home_team','away_team','correct_score','1N2_1','1N2_N','1N2_2','betexplorer']
df.drop_duplicates(inplace=True)
df.to_csv(f'data/urls_{timestampStr}.csv', index=False)
df.to_csv(f'data/oddsportal_urls_{timestampStr}.csv')

"""
for idx, r in tqdm(df.iterrows()):
    odp = f'https://www.oddsportal.com/{r.oddsportal}'
    page = requests.get(odp)

    content = page.content
    soup = BeautifulSoup(page.content, 'html.parser')
    p_class = soup.find_all('p', class_='result')
    #df.loc[idx,'result']=p_class
"""

print(f'Rows scrapped {df.shape}')
df.sample(8)

for idx, r in df.iterrows():
    d=r.matchday
    if len(d)<10:
        df.loc[idx,'matchday']=f'{d}2022'
df['matchdate'] = pd.to_datetime(df['matchday'],  format="%d.%m.%Y")
df['period']=df['matchdate'].dt.to_period('M')
df.drop_duplicates(['matchday','home_team','away_team'], inplace=True)
df.shape



for idx, r in tqdm(df.iterrows()):
    tags = r.oddsportal.split('/')
    df.loc[idx,'location']=tags[1]
    df.loc[idx,'season']=tags[2].split('-')[-1]
    l=tags[2].split('-')[:-1]
    df.loc[idx,'competition']='-'.join(l)
    # compute 1/N/2
    scores = r.correct_score
    score = scores.split(':')
    df.loc[idx,'home_score']=int(score[0])
    df.loc[idx,'away_score']=int(score[1])
    if score[0]>score[1]:
        df.loc[idx,'bet_won']='1'    
        df.loc[idx,'target']=0
        df.loc[idx,'odd_win']=r['1N2_1']    
        if r['1N2_1']<r['1N2_N'] and r['1N2_1']<r['1N2_2']:
            df.loc[idx,'best_odd_won']=True
        else:
            df.loc[idx,'best_odd_won']=False
    elif score[0]<score[1]:
        df.loc[idx,'bet_won']='2'
        df.loc[idx,'target']=2
        df.loc[idx,'odd_win']=r['1N2_2'] 
        if r['1N2_2']<r['1N2_N'] and r['1N2_2']<r['1N2_1']:
            df.loc[idx,'best_odd_won']=True
        else:
            df.loc[idx,'best_odd_won']=False

    else:
        df.loc[idx,'bet_won']='N'
        df.loc[idx,'target']=1       
        df.loc[idx,'odd_win']=r['1N2_N'] 
        if r['1N2_N']<r['1N2_1'] and r['1N2_N']<r['1N2_2']:
            df.loc[idx,'best_odd_won']=True
        else:
            df.loc[idx,'best_odd_won']=False

    
try:
    df.drop('matchdate', inplace=True)
    # Convert multiple columns

except:
    pass

df = df.astype({'1N2_1':'float','1N2_N':'float','1N2_2':'float','odd_win':'float','best_odd_won':'boolean'})    
df.drop_duplicates(inplace=True)
df.drop(['oddsportal','betexplorer','matchday'],axis=1).drop_duplicates().to_csv(f'data/20-years-odds-{timestampStr}.csv', index=False)

df.head(8)

100%|██████████| 8/8 [00:12<00:00,  1.59s/it]


Rows scrapped (3310, 9)


3310it [00:00, 4130.58it/s]


,oddsportal,matchday,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,betexplorer,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,soccer/england/premier-league-2021-2022/arsena...,22.05.2022,Arsenal,Everton,5:1,1.34,5.50,8.98,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,5.0,1.0,1,0.0,1.34,True
1,soccer/england/premier-league-2021-2022/brentf...,22.05.2022,Brentford,Leeds,1:2,2.32,3.79,2.88,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,1.0,2.0,2,2.0,2.88,False
2,soccer/england/premier-league-2021-2022/bright...,22.05.2022,Brighton,West Ham,3:1,2.66,3.46,2.65,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,3.0,1.0,1,0.0,2.66,False
3,soccer/england/premier-league-2021-2022/burnle...,22.05.2022,Burnley,Newcastle,1:2,2.47,3.27,3.01,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,1.0,2.0,2,2.0,3.01,False
4,soccer/england/premier-league-2021-2022/chelse...,22.05.2022,Chelsea,Watford,2:1,1.19,7.41,15.39,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,2.0,1.0,1,0.0,1.19,True
5,soccer/england/premier-league-2021-2022/crysta...,22.05.2022,Crystal Palace,Manchester Utd,1:0,2.81,3.47,2.50,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,1.0,0.0,1,0.0,2.81,False
6,soccer/england/premier-league-2021-2022/leices...,22.05.2022,Leicester,Southampton,4:1,1.70,4.26,4.54,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,4.0,1.0,1,0.0,1.70,True
7,soccer/england/premier-league-2021-2022/liverp...,22.05.2022,Liverpool,Wolves,3:1,1.13,9.19,19.26,https://www.betexplorer.com/soccer/england/pre...,2022-05-22,2022-05,england,2022,premier-league-2021,3.0,1.0,1,0.0,1.13,True
